[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jrkasprzyk/CVEN5393/blob/main/streamflow_nowak.ipynb)

*This notebook is part of course notes for CVEN 5393: Water Resource Systems and Management, by Prof. Joseph Kasprzyk at CU Boulder.*

This notebook implements the method published below:

Nowak, K., J. Prairie, B. Rajagopalan, and U. Lall (2010), A nonparametric stochastic approach for multisite
disaggregation of annual to daily streamflow, Water Resour. Res., 46, W08529. [DOI](https://doi.org/10.1029/2009WR008530).

In [1]:
import pandas as pd #for dataframes and data processing
import numpy as np #for numerical computation
import matplotlib.pyplot as plt #for plotting
import sys #system functions
from scipy import interpolate #bring in only the interpolate function
import plotly.express as px #plotly express for fast interactive plotting
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [32]:
def simulate_flow(rng, Z_input, p_input, years_input, sim_Z):

  #inputs:
  #rng - random number generator instance
  #Z_input - sequence of aggregated flows
  #p_input - 2d array of proportion vectors from historical record
  #years_input - sequence of historical years
  #sim_Z - simulated aggregated flow

  #return: sequence of simulated, disaggregated flow

  # Calculate the distance between the yearly flows and the
  # simulated value
  dist = np.absolute(Z_input - sim_Z)

  # inds will be the indices of the original sequence in ascending order
  inds = dist.argsort()

  # these are the yearly flows, sorted by their distance from the simulated flow
  sorted_Z = Z_input[inds]

  # the number of neighbors is a function of the
  # number of datapoints in the yearly sequence
  K = int(np.floor(np.sqrt(len(Z_input))))

  # the weight function gives the most weight
  # to the first neighbor (eq 1 in the paper)
  W = np.zeros(K)
  for i in range(K):
    W[i] = (1./(i+1.))/sum(1./k for k in range(1, K+1))

  # here, we only keep the K nearest neighbors based on distance
  neighbors = sorted_Z[0:K]
  neighbors_inds = inds[0:K]

  # the index of the closest year...
  chosen_index = rng.choice(neighbors_inds, size=1, p=W)

  # ...is used to find a proportion vector
  sim_p = p_input[chosen_index, :]

  # the simulated flow sequence is the proportion multiplied by simulated yearly flow
  sim_flow = sim_p * sim_Z

  print(f"Sim annual {sim_Z: 0.1f}, using analog year {str(*years_input[chosen_index])}: {np.array2string(sim_flow, precision=1, floatmode='fixed')}")

  return sim_flow

# Example 1: Simulating seasonal flow, 1967-1975 (from the paper)

In [33]:
ex1_rng = np.random.default_rng(seed=42)

ex1_obs_years = np.array([1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975])
ex1_obs_ann_flow = np.array([35., 40., 33., 52., 43., 56., 38., 49., 32.])
ex1_obs_p = np.array(
    [[.1, .3, .4, .2],
     [.15, .25, .35, .25],
     [.1, .2, .5, .2],
     [.5, .15, .65, .15],
     [.2, .2, .4, .2],
     [.1, .2, .4, .3],
     [.15, .2, .4, .25],
     [.05, .1, .8, .05],
     [.2, .2, .5, .1]
     ]
    )

Below, we perform the simulation 10 different times. The algorithm has one component of randomness: a random number is used to generate which analog year is chosen. Therefore, repeating the simulation multiple times will create new timeseries, since different analog years have different proportion vectors.

In [34]:
ex1_sim_ann_flow = 70.

# 10 different simulations for the same annual flow
for i in range(10):
  ex1_sim_seas_flow = simulate_flow(ex1_rng, ex1_obs_ann_flow, ex1_obs_p, ex1_obs_years, ex1_sim_ann_flow)

Sim annual  70.0, using analog year 1970: [[35.0 10.5 45.5 10.5]]
Sim annual  70.0, using analog year 1972: [[ 7.0 14.0 28.0 21.0]]
Sim annual  70.0, using analog year 1974: [[ 3.5  7.0 56.0  3.5]]
Sim annual  70.0, using analog year 1970: [[35.0 10.5 45.5 10.5]]
Sim annual  70.0, using analog year 1972: [[ 7.0 14.0 28.0 21.0]]
Sim annual  70.0, using analog year 1974: [[ 3.5  7.0 56.0  3.5]]
Sim annual  70.0, using analog year 1970: [[35.0 10.5 45.5 10.5]]
Sim annual  70.0, using analog year 1970: [[35.0 10.5 45.5 10.5]]
Sim annual  70.0, using analog year 1972: [[ 7.0 14.0 28.0 21.0]]
Sim annual  70.0, using analog year 1972: [[ 7.0 14.0 28.0 21.0]]


# Example 2: Seasonal flow with reservoir simulation

*This is a new example for Spring 2024, which maintains four timesteps per year but uses more data with more questions to answer. This is also an illustrative dataset only.*

A particular river has observed seasonal streamflow (af) from 2000-2007. A climate model has been used to create 5 different synthetic sequences of new annual flows, each of which is 3 years long.

Use the Nowak et al disaggregation method to create synthetic sequences of 3 years' worth of seasonal data. For each annual sequence provided, repeat the process 2 times to create random disaggregated seuqences from each annual sequence.

When you're done, you should have 10 new seasonal sequences!


In [53]:
ex2_obs_years = np.array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007])

ex2_obs_seas_flow = np.array([50., 52., 80., 105., 60., 70., 90., 110.,
                              60., 24., 40., 80., 40., 30., 80., 92.,
                              81., 55., 81., 92., 50., 60., 100., 150.,
                              20., 10., 80., 100., 50., 20., 30., 120.])

# shape: sequences in rows; years in columns
ex2_sim_ann_flow = np.array([[275., 300., 305],
                             [200., 220., 400.],
                             [300., 310., 289.],
                             [260., 311., 400.],
                             [200., 250., 260.]])

In this example, all we have started with is the seasonal flow, in other words this is the observed disaggregated flow. We must calculate the proportion vector as well as find the observed annual flow.

We will also be performing the calculations over multiple timesteps, using three different sequences of simulated annual data.

In [54]:
# create a matrix with 8 rows (years) and 4 columns (seasons)
ex2_obs_seas_flow_mat = ex2_obs_seas_flow.reshape((8, 4))

# find the sum of each row, which is the observed annual flow
ex2_obs_ann_flow = np.sum(ex2_obs_seas_flow_mat, axis=1)

# prepare the variable for the proportion vector
ex2_obs_p = np.zeros((8, 4))

# calculate the proportion vector
for i in range(4):
  ex2_obs_p[:, i] = ex2_obs_seas_flow_mat[:, i]/ex2_obs_ann_flow

Now the observed data is the same format as Example 1. When running the calculations for this problem, though, we will need to store them in a larger format since we are doing one calculation for every simulated annual flow, and repeating it with multiple replicates.

In [58]:
# reset the random number generator to produce consistent results for this example
ex2_rng = np.random.default_rng(seed=42)

# prepare results matrix:
# rows: seasons in rows (4 seasons, 3 years long = 12 rows); simulations in columns
ex2_sim_seas_flow = np.zeros((12, 10))

j = 0 #column for the final output
for r in range(2): # repeat for 2 replicates
  for s in range(5): # multiple sequences
    i = 0 # row for the final output
    for y in range(3): # multiple years
      sim_seas = simulate_flow(ex2_rng, ex2_obs_ann_flow, ex2_obs_p, ex2_obs_years,
                               ex2_sim_ann_flow[s, y])
      ex2_sim_seas_flow[i:i+4, j] = sim_seas
      i = i+4
    j = j+1


Sim annual  275.0, using analog year 2003: [[ 45.5  34.1  90.9 104.5]]
Sim annual  300.0, using analog year 2004: [[78.6 53.4 78.6 89.3]]
Sim annual  305.0, using analog year 2000: [[ 53.1  55.3  85.0 111.6]]
Sim annual  200.0, using analog year 2006: [[19.0  9.5 76.2 95.2]]
Sim annual  220.0, using analog year 2007: [[ 50.0  20.0  30.0 120.0]]
Sim annual  400.0, using analog year 2001: [[ 72.7  84.8 109.1 133.3]]
Sim annual  300.0, using analog year 2000: [[ 52.3  54.4  83.6 109.8]]
Sim annual  310.0, using analog year 2001: [[ 56.4  65.8  84.5 103.3]]
Sim annual  289.0, using analog year 2000: [[ 50.3  52.4  80.6 105.7]]
Sim annual  260.0, using analog year 2003: [[43.0 32.2 86.0 98.8]]
Sim annual  311.0, using analog year 2004: [[81.5 55.4 81.5 92.6]]
Sim annual  400.0, using analog year 2001: [[ 72.7  84.8 109.1 133.3]]
Sim annual  200.0, using analog year 2002: [[58.8 23.5 39.2 78.4]]
Sim annual  250.0, using analog year 2007: [[ 56.8  22.7  34.1 136.4]]
Sim annual  260.0, using a

In [69]:
my_df = pd.DataFrame(ex2_sim_seas_flow)

In [73]:
my_df

,0,1,2,3,4,5,6,7,8,9
0,45.454545,19.047619,52.264808,42.975207,58.823529,47.909408,19.047619,78.640777,45.296167,58.823529
1,34.090909,9.523810,54.355401,32.231405,23.529412,49.825784,9.523810,53.398058,47.108014,23.529412
2,90.909091,76.190476,83.623693,85.950413,39.215686,76.655052,76.190476,78.640777,72.473868,39.215686
3,104.545455,95.238095,109.756098,98.842975,78.431373,100.609756,95.238095,89.320388,95.121951,78.431373
4,78.640777,50.000000,56.363636,81.524272,56.818182,78.640777,50.000000,56.363636,81.524272,41.322314
5,53.398058,20.000000,65.757576,55.355987,22.727273,53.398058,20.000000,65.757576,55.355987,30.991736
6,78.640777,30.000000,84.545455,81.524272,34.090909,78.640777,30.000000,84.545455,81.524272,82.644628
7,89.320388,120.000000,103.333333,92.595469,136.363636,89.320388,120.000000,103.333333,92.595469,95.041322
8,53.135889,72.727273,50.348432,72.727273,42.975207,79.951456,72.727273,75.757282,55.555556,45.296167
9,55.261324,84.848485,52.362369,84.848485,32.231405,54.288026,84.848485,51.440129,66.666667,47.108014
